# Simple transformers

## Install packages

In [1]:
!pip install -q condacolab
!conda create -n st python pandas tqdm
!conda activate st
!conda install pytorch>=1.6 cudatoolkit=11.0 -c pytorch
!pip install simpletransformers

Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | done

## Package Plan ##

  environment location: /usr/local/envs/st

  added / updated specs:
    - pandas
    - python
    - tqdm


The following NEW packages will be INSTALLED:

  _libgcc_mutex      conda-forge/linux-64::_libgcc_mutex-0.1-conda_forge 
  _openmp_mutex      conda-forge/linux-64::_openmp_mutex-4.5-2_gnu 
  bzip2              conda-forge/linux-64::bzip2-1.0.8-h7f98852_4 
  ca-certificates    conda-forge/linux-64::ca-certificates-2022.12.7-ha878542_0 
  colorama           conda-forge/noarch::colorama-0.4.6-pyhd8ed1ab_0 
  ld_impl_linux-64   conda-forge/linux-64::ld_impl_linux-64-2.40-h41732ed_0 
  libblas            conda-forge/linux-64::libblas-3.9.0-16_linux64_openblas 
  libcblas           conda-forge/linux-64::libcblas-3.9.0-16_linux64_openblas 
  libffi    

In [2]:
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


# Getting data

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
!unzip './drive/MyDrive/p6.zip'
drive.flush_and_unmount()

Archive:  ./drive/MyDrive/p6.zip
replace test_data.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [1]:
import pandas as pd
import torch
import numpy as np

if torch.cuda.is_available():
  torch.cuda.empty_cache()
  device = torch.device('cuda')
SEED = 128412894
torch.manual_seed(SEED)
torch.cuda.is_available()

False

In [5]:
TRAIN_TO_VALID = 0.8
batch_size = 128

train_data = pd.read_csv("./data/train_data.csv").sample(frac=1)

valid_index = int(TRAIN_TO_VALID * len(train_data))
trainset, validset = train_data[:valid_index], train_data[valid_index:]

In [6]:
print(len(trainset))
print(len(validset))

13113
3279


# Classification model

### Weight data because of class imbalance

In [7]:
train_data['rating'].value_counts()

4    7243
3    4831
2    1747
1    1434
0    1137
Name: rating, dtype: int64

In [8]:
WEIGHTS = [1, 2, 4, 4, 4]

In [10]:
from simpletransformers.classification import ClassificationModel

# Create a ClassificationModel
model = ClassificationModel('roberta', 'roberta-base', num_labels=5, weight=WEIGHTS)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

In [11]:
model.train_model(trainset, output_dir="outputs2", valid_df=validset)

/usr/local/lib/python3.8/site-packages/simpletransformers/classification/classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/13113 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/1640 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

## Evaluate model

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(validset)

In [ ]:
print(len(wrong_predictions)/len(model_outputs))

## Save predicted test results

## Purge testset

In [ ]:
test_data = pd.read_csv("./data/test_data.csv", header=None)
test_data = test_data.values.tolist()

In [ ]:
predictions, raw_outputs = model.predict(test_data)
predictions = pd.DataFrame(predictions)
predictions.to_csv('output.csv', index=False, header=False)